## New Standardization Workflow

In [6]:
import pandas as pd

df = pd.read_csv('NW Publisher – Address Export - 20250722 - complete.csv')

# Filter and clean
clean = df[df['Status'] != 'NewFromPublisher']

# Deduplicate by Number + Street + PostalCode
clean_unique = clean.drop_duplicates(subset=['Number','Street','PostalCode'], keep='first')

# Save
out = 'cleaned_addresses.csv'
clean_unique.to_csv(out, index=False)
out


'cleaned_addresses.csv'

In [ ]:
import os
import time
import json
import requests
import pandas as pd
from datetime import datetime

# --- CONFIGURE ---



AUTH_ID = "c158cf93-8a4d-c43c-dade-f372cbeac850"   # replace if needed
AUTH_TOKEN = "e7UMwEf0GevqQpxwdLaO"                # replace if needed
SMARTY_URL = "https://us-street.api.smarty.com/street-address"

INPUT_CSV = os.path.join("cleaned_sample.csv")  # input file (expects Number/Street/Suburb/State/PostalCode or similar)
CACHE_DIR = os.path.join("json_cache")
os.makedirs(CACHE_DIR, exist_ok=True)

# candidate names for unit/apartment in input
SECONDARY_CANDIDATES = [
    "ApartmentNumber", "Apartment", "Apt", "Unit", "Secondary", "SecondaryUnit",
    "apt_number", "apt", "unit_number"
]

def get_input_secondary(row):
    for k in SECONDARY_CANDIDATES:
        if k in row.index:
            v = row.get(k)
            if pd.notna(v) and str(v).strip():
                return str(v).strip()
    return ""

def safe_filename(s):
    return "".join(c if c.isalnum() or c in "._- " else "_" for c in str(s)).strip().replace(" ", "_")

def save_response(name_base, data, meta):
    path = os.path.join(CACHE_DIR, f"{name_base}.json")
    meta_path = os.path.join(CACHE_DIR, f"{name_base}_meta.json")
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, indent=2)

def main():
    df = pd.read_csv(INPUT_CSV)
    for i, row in df.iterrows():
        number = str(row.get("Number", "")).strip()
        street_raw = str(row.get("Street", "")).strip()
        street = f"{number} {street_raw}".strip()
        city = str(row.get("Suburb", "")).strip()
        state = str(row.get("State", "CA")).strip()
        zipcode = str(row.get("PostalCode", "")).replace(".0", "").strip()
        input_secondary = get_input_secondary(row)

        params = {
            "auth-id": AUTH_ID,
            "auth-token": AUTH_TOKEN,
            "street": street,
            "city": city,
            "state": state,
            "zipcode": zipcode,
        }
        if input_secondary:
            params["secondary"] = input_secondary

        # best-effort filename check: if response cached by input row (index) skip
        tentative_name = f"row_{i}"
        meta_stub = {"row_index": int(i), "sent_params": params, "input_secondary": input_secondary, "timestamp": datetime.utcnow().isoformat() + "Z"}

        # Do request
        try:
            r = requests.get(SMARTY_URL, params=params, timeout=15)
            # try parse JSON even on non-200 to capture error responses in cache
            data = r.json() if r.content else None
            # compute name base from delivery_point_barcode if possible
            name_base = tentative_name
            if isinstance(data, list) and len(data) > 0 and isinstance(data[0], dict):
                dpb = data[0].get("delivery_point_barcode") or data[0].get("metadata", {}).get("delivery_point_barcode")
                if dpb:
                    name_base = safe_filename(dpb)
                else:
                    # try primary number + zipcode for nicer name
                    comps = data[0].get("components", {})
                    pn = comps.get("primary_number") or number or str(i)
                    z = comps.get("zipcode") or zipcode or ""
                    name_base = safe_filename(f"{pn}_{z}_{i}")
            else:
                # fallback name includes street and index
                name_base = safe_filename(f"{street_raw}_{i}")[:120] or tentative_name

            save_response(name_base, data, meta_stub | {"status_code": r.status_code})
            print(f"[{i}] saved cache: {name_base}.json (HTTP {r.status_code})")

            # Respect rate limit a bit
            time.sleep(0.5)

        except Exception as e:
            # save error meta for debugging
            err_meta = meta_stub | {"error": str(e)}
            err_name = safe_filename(f"error_row_{i}")
            with open(os.path.join(CACHE_DIR, f"{err_name}_meta.json"), "w", encoding="utf-8") as f:
                json.dump(err_meta, f, indent=2)
            print(f"[{i}] Exception: {e}")

if __name__ == "__main__":
    main()

[0] saved cache: 903052323120.json (HTTP 200)
[1] saved cache: 903052323166.json (HTTP 200)
[2] saved cache: 903052346170.json (HTTP 200)
[3] saved cache: 903052346215.json (HTTP 200)


In [1]:
import os
import json
import pandas as pd
from glob import glob

CACHE_DIR = os.path.join("json_cache")
OUT_CSV = os.path.join("output", "cached_parsed_addresses.csv")
os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)

def parse_file(path):
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
    except Exception:
        return None

    # If data is a list, take first candidate (common Smarty response)
    entry = data[0] if isinstance(data, list) and len(data) > 0 else (data if isinstance(data, dict) else None)
    if not entry:
        return None

    comps = entry.get("components", {})
    meta = entry.get("metadata", {})
    analysis = entry.get("analysis", {})
    row = {
        "cache_file": os.path.basename(path),
        "delivery_line_1": entry.get("delivery_line_1", ""),
        "delivery_line_2": entry.get("delivery_line_2", ""),
        "last_line": entry.get("last_line", ""),
        "delivery_point_barcode": entry.get("delivery_point_barcode", "") or meta.get("delivery_point_barcode", ""),
        # components
        "primary_number": comps.get("primary_number", ""),
        "street_predirection": comps.get("street_predirection", ""),
        "street_name": comps.get("street_name", ""),
        "street_suffix": comps.get("street_suffix", ""),
        "secondary_designator": comps.get("secondary_designator", ""),
        "secondary_number": comps.get("secondary_number", ""),
        "city_name": comps.get("city_name", "") or comps.get("default_city_name", ""),
        "state_abbreviation": comps.get("state_abbreviation", ""),
        "zipcode": comps.get("zipcode", ""),
        "plus4_code": comps.get("plus4_code", ""),
        "delivery_point": comps.get("delivery_point", ""),
        "delivery_point_check_digit": comps.get("delivery_point_check_digit", ""),
        
        # metadata/location
        "latitude": meta.get("latitude", ""),
        "longitude": meta.get("longitude", ""),
        "precision": meta.get("precision", ""),
        # analysis/dpv
        "dpv_match_code": analysis.get("dpv_match_code", ""),
        "dpv_footnotes": analysis.get("dpv_footnotes", ""),
        "active": analysis.get("active", ""),
    }
    return row

def main():
    files = sorted(glob(os.path.join(CACHE_DIR, "*.json")))
    # ignore meta companion files if any (we included *_meta.json)
    files = [f for f in files if not f.endswith("_meta.json")]
    rows = []
    for fpath in files:
        parsed = parse_file(fpath)
        if parsed:
            rows.append(parsed)
    if not rows:
        print("No parsed rows found in cache.")
        return
    df = pd.DataFrame(rows)
    # add a composed StandardizedAddress column using delivery lines when available
    def compose_std(r):
        parts = []
        if r.get("delivery_line_1"):
            parts.append(r["delivery_line_1"])
        if r.get("delivery_line_2"):
            parts.append(r["delivery_line_2"])
        if r.get("last_line"):
            parts.append(r["last_line"])
        return ", ".join([p for p in parts if p])
    df["StandardizedAddress"] = df.apply(compose_std, axis=1)
    df.to_csv(OUT_CSV, index=False)
    print("Saved parsed cached data to:", OUT_CSV)

if __name__ == "__main__":
    main()

Saved parsed cached data to: output\cached_parsed_addresses.csv


Monitors remaining lookups

In [3]:
import os
import time
import json
import requests
import pandas as pd
from datetime import datetime

# --- CONFIGURE ---
AUTH_ID = "c158cf93-8a4d-c43c-dade-f372cbeac850"
AUTH_TOKEN = "e7UMwEf0GevqQpxwdLaO"
SMARTY_URL = "https://us-street.api.smarty.com/street-address"

INPUT_CSV = os.path.join("cleaned_addresses_input.csv")
CACHE_DIR = os.path.join("json_cache")
os.makedirs(CACHE_DIR, exist_ok=True)

SECONDARY_CANDIDATES = [
    "ApartmentNumber", "Apartment", "Apt", "Unit", "Secondary", "SecondaryUnit",
    "apt_number", "apt", "unit_number"
]

def get_input_secondary(row):
    for k in SECONDARY_CANDIDATES:
        if k in row.index:
            v = row.get(k)
            if pd.notna(v) and str(v).strip():
                return str(v).strip()
    return ""

def safe_filename(s):
    return "".join(c if c.isalnum() or c in "._- " else "_" for c in str(s)).strip().replace(" ", "_")

def save_response(name_base, data, meta):
    path = os.path.join(CACHE_DIR, f"{name_base}.json")
    meta_path = os.path.join(CACHE_DIR, f"{name_base}_meta.json")
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, indent=2)

def main():
    df = pd.read_csv(INPUT_CSV)

    for i, row in df.iterrows():
        number = str(row.get("Number", "")).strip()
        street_raw = str(row.get("Street", "")).strip()
        street = f"{number} {street_raw}".strip()
        city = str(row.get("Suburb", "")).strip()
        state = str(row.get("State", "CA")).strip()
        zipcode = str(row.get("PostalCode", "")).replace(".0", "").strip()
        input_secondary = get_input_secondary(row)

        params = {
            "auth-id": AUTH_ID,
            "auth-token": AUTH_TOKEN,
            "street": street,
            "city": city,
            "state": state,
            "zipcode": zipcode,
        }
        if input_secondary:
            params["secondary"] = input_secondary

        tentative_name = f"row_{i}"
        meta_stub = {
            "row_index": int(i),
            "sent_params": params,
            "input_secondary": input_secondary,
            "timestamp": datetime.utcnow().isoformat() + "Z"
        }

        try:
            r = requests.get(SMARTY_URL, params=params, timeout=15)
            data = r.json() if r.content else None

            # --- NEW: CHECK REMAINING LOOKUPS ---
            remaining = r.headers.get("X-Rate-Limit-Remaining")
            if remaining is not None:
                try:
                    remaining = int(remaining)
                    print(f"Remaining lookups: {remaining}")
                    if remaining <= 0:
                        print(f"⚠️ No more lookups left. Stopped at row {i}.")
                        return
                except ValueError:
                    print("Warning: Could not parse remaining lookup header.")

            # compute name base
            name_base = tentative_name
            if isinstance(data, list) and len(data) > 0 and isinstance(data[0], dict):
                dpb = data[0].get("delivery_point_barcode") or data[0].get("metadata", {}).get("delivery_point_barcode")
                if dpb:
                    name_base = safe_filename(dpb)
                else:
                    comps = data[0].get("components", {})
                    pn = comps.get("primary_number") or number or str(i)
                    z = comps.get("zipcode") or zipcode or ""
                    name_base = safe_filename(f"{pn}_{z}_{i}")
            else:
                name_base = safe_filename(f"{street_raw}_{i}")[:120] or tentative_name

            save_response(name_base, data, meta_stub | {"status_code": r.status_code})
            print(f"[{i}] saved cache: {name_base}.json (HTTP {r.status_code})")

            time.sleep(0.5)

        except Exception as e:
            err_meta = meta_stub | {"error": str(e)}
            err_name = safe_filename(f"error_row_{i}")
            with open(os.path.join(CACHE_DIR, f"{err_name}_meta.json"), "w", encoding="utf-8") as f:
                json.dump(err_meta, f, indent=2)
            print(f"[{i}] Exception: {e}")

    print("🎉 Completed all rows successfully!")

if __name__ == "__main__":
    main()


[0] saved cache: 903052323247.json (HTTP 200)
[1] saved cache: 903052346251.json (HTTP 200)
[2] saved cache: 903052325001.json (HTTP 200)
[3] saved cache: 903052324011.json (HTTP 200)
[4] saved cache: 903052325047.json (HTTP 200)
[5] saved cache: 903052324057.json (HTTP 200)
[6] saved cache: 903053333993.json (HTTP 200)
[7] saved cache: 903052324093.json (HTTP 200)
[8] saved cache: 903052325128.json (HTTP 200)
[9] saved cache: 903052324156.json (HTTP 200)
[10] saved cache: 903052325164.json (HTTP 200)
[11] saved cache: 903052324174.json (HTTP 200)
[12] saved cache: 903052325209.json (HTTP 200)
[13] saved cache: 903052324219.json (HTTP 200)
[14] saved cache: 903052301115.json (HTTP 200)
[15] saved cache: 903052345991.json (HTTP 200)
[16] saved cache: 903052302169.json (HTTP 200)
[17] saved cache: 903052301179.json (HTTP 200)
[18] saved cache: 903052302204.json (HTTP 200)
[19] saved cache: 903052302240.json (HTTP 200)
[20] saved cache: 903052301250.json (HTTP 200)
[21] saved cache: 90305

KeyboardInterrupt: 

In [4]:
import os
import time
import json
import requests
import pandas as pd
from datetime import datetime

# --- CONFIGURE ---
AUTH_ID = "fc20c9c5-366d-71ae-5c8f-da11aa762646"
AUTH_TOKEN = "rOOG4cFXIaOANVasFnu7"
SMARTY_URL = "https://us-street.api.smarty.com/street-address"

INPUT_CSV = "remaining_cleaned_addresses_input.csv"
CACHE_DIR = "json_cache"
LOG_FILE = "process_log.csv"
REMAINING_CSV = "remaining_rows.csv"
INVALID_CSV = "invalid_rows.csv"   # NEW: store rows with invalid JSON responses

os.makedirs(CACHE_DIR, exist_ok=True)

SECONDARY_CANDIDATES = [
    "ApartmentNumber", "Apartment", "Apt", "Unit", "Secondary", "SecondaryUnit",
    "apt_number", "apt", "unit_number"
]

# ========= Helper Functions ========= #

def get_input_secondary(row):
    for k in SECONDARY_CANDIDATES:
        if k in row.index:
            v = row.get(k)
            if pd.notna(v) and str(v).strip():
                return str(v).strip()
    return ""

def safe_filename(s):
    return "".join(c if c.isalnum() or c in "._- " else "_" for c in str(s)).strip().replace(" ", "_")

def cached_rows():
    done = set()
    for f in os.listdir(CACHE_DIR):
        if f.endswith("_meta.json") and f.startswith("row_"):
            try:
                idx = int(f.split("_")[1].split(".")[0])
                done.add(idx)
            except:
                pass
    return done

def append_log(row_index, street, status, filename, http_code, error_message=""):
    entry = {
        "timestamp": datetime.utcnow().isoformat() + "Z",
        "row_index": row_index,
        "street": street,
        "status": status,
        "filename": filename,
        "http_code": http_code,
        "error_message": error_message
    }
    df = pd.DataFrame([entry])
    df.to_csv(LOG_FILE, mode="a", index=False, header=not os.path.exists(LOG_FILE))

def save_response(name_base, data, meta):
    with open(os.path.join(CACHE_DIR, f"{name_base}.json"), "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)
    with open(os.path.join(CACHE_DIR, f"{name_base}_meta.json"), "w", encoding="utf-8") as f:
        json.dump(meta, f, indent=2)

# ========= MAIN PROCESSING ========= #

def main():
    df = pd.read_csv(INPUT_CSV)

    # NEW ======================================================
    invalid_rows = []  # collect invalid rows here
    # ==========================================================

    done_rows = cached_rows()
    print(f"Found {len(done_rows)} previously processed rows. Resuming automatically…")

    total_rows = len(df)

    for i, row in df.iterrows():
        if i in done_rows:
            print(f"[{i}] Skipping (cached).")
            continue

        number = str(row.get("Number", "")).strip()
        street_raw = str(row.get("Street", "")).strip()
        street = f"{number} {street_raw}".strip()
        city = str(row.get("Suburb", "")).strip()
        state = str(row.get("State", "CA")).strip()
        zipcode = str(row.get("PostalCode", "")).replace(".0", "").strip()
        input_secondary = get_input_secondary(row)

        params = {
            "auth-id": AUTH_ID,
            "auth-token": AUTH_TOKEN,
            "street": street,
            "city": city,
            "state": state,
            "zipcode": zipcode,
        }
        if input_secondary:
            params["secondary"] = input_secondary

        tentative_name = f"row_{i}"
        meta_stub = {
            "row_index": int(i),
            "sent_params": params,
            "timestamp": datetime.utcnow().isoformat() + "Z",
        }

        # --- RETRY MECHANISM ---
        retries = 3
        data = None
        while retries > 0:
            try:
                r = requests.get(SMARTY_URL, params=params, timeout=15)
                data = r.json() if r.content else None
                break
            except requests.exceptions.Timeout:
                retries -= 1
                wait = (4 - retries)
                print(f"[{i}] Timeout. Retrying in {wait}s…")
                time.sleep(wait)
        else:
            print(f"[{i}] FAILED after retries.")
            append_log(i, street, "timeout_failure", tentative_name, "TIMEOUT")
            continue

        # --- EXHAUSTION OR ERROR CHECK ---
        exhausted = False

        if r.status_code == 402:
            exhausted = True

        if isinstance(data, dict) and "errors" in data:
            exhausted = True

        if exhausted:
            print(f"❌ API exhausted at row {i}. Saving remaining rows to CSV…")
            remaining_df = df.iloc[i:]
            remaining_df.to_csv(REMAINING_CSV, index=False)

            # ALSO save invalid rows collected so far
            if invalid_rows:
                pd.DataFrame(invalid_rows).to_csv(INVALID_CSV, index=False)

            return

        # --- VALIDITY CHECK (NEW SECTION) ---
        if not isinstance(data, list) or len(data) == 0:
            print(f"[{i}] ⚠ INVALID JSON response → storing to invalid_rows.csv")
            invalid_rows.append(row)
            append_log(i, street, "invalid_json", tentative_name, r.status_code, "Empty or invalid JSON")
            continue

        # --- NAME FOR CACHE ---
        name_base = tentative_name
        first = data[0]

        dpb = first.get("delivery_point_barcode") or first.get("metadata", {}).get("delivery_point_barcode")
        if dpb:
            name_base = safe_filename(dpb)
        else:
            comps = first.get("components", {})
            pn = comps.get("primary_number") or number or i
            z = comps.get("zipcode") or zipcode
            name_base = safe_filename(f"{pn}_{z}_{i}")

        # Save the response JSON + meta
        save_response(name_base, data, meta_stub | {"status_code": r.status_code})
        append_log(i, street, "success", name_base, r.status_code)

        print(f"[{i}] Saved: {name_base}.json (HTTP {r.status_code})")

        time.sleep(0.5)

    # AFTER LOOP: save invalid rows
    if invalid_rows:
        pd.DataFrame(invalid_rows).to_csv(INVALID_CSV, index=False)

    print("🎉 Completed all rows successfully!")

if __name__ == "__main__":
    main()


Found 0 previously processed rows. Resuming automatically…
[0] ⚠ INVALID JSON response → storing to invalid_rows.csv
[1] ⚠ INVALID JSON response → storing to invalid_rows.csv
[2] ⚠ INVALID JSON response → storing to invalid_rows.csv
[3] ⚠ INVALID JSON response → storing to invalid_rows.csv
[4] ⚠ INVALID JSON response → storing to invalid_rows.csv
[5] ⚠ INVALID JSON response → storing to invalid_rows.csv
[6] Saved: 903052218019.json (HTTP 200)
[7] Saved: 903052218055.json (HTTP 200)
[8] Saved: 903052218091.json (HTTP 200)
[9] Saved: 903052218136.json (HTTP 200)
[10] Saved: 903052218172.json (HTTP 200)
[11] Saved: 903052218217.json (HTTP 200)
[12] Saved: 903052218253.json (HTTP 200)
[13] Saved: 903053653998.json (HTTP 200)
[14] ⚠ INVALID JSON response → storing to invalid_rows.csv
[15] Saved: 903052220043.json (HTTP 200)
[16] Saved: 903052220089.json (HTTP 200)
[17] Saved: 903052220124.json (HTTP 200)
[18] Saved: 903052220160.json (HTTP 200)
[19] Saved: 903052220205.json (HTTP 200)
[20] 

In [6]:
import os
import time
import json
import requests
import pandas as pd
from datetime import datetime

# --- CONFIGURE ---
AUTH_ID = "fc20c9c5-366d-71ae-5c8f-da11aa762646"
AUTH_TOKEN = "rOOG4cFXIaOANVasFnu7"
SMARTY_URL = "https://us-street.api.smarty.com/street-address"

INPUT_CSV = "remaining_rows0.csv"
CACHE_DIR = "json_cache"
LOG_FILE = "process_log.csv"
REMAINING_CSV = "remaining_rows.csv"
INVALID_CSV = "invalid_rows.csv"   # NEW: store rows with invalid JSON responses

os.makedirs(CACHE_DIR, exist_ok=True)

SECONDARY_CANDIDATES = [
    "ApartmentNumber", "Apartment", "Apt", "Unit", "Secondary", "SecondaryUnit",
    "apt_number", "apt", "unit_number"
]

# ========= Helper Functions ========= #

def get_input_secondary(row):
    for k in SECONDARY_CANDIDATES:
        if k in row.index:
            v = row.get(k)
            if pd.notna(v) and str(v).strip():
                return str(v).strip()
    return ""

def safe_filename(s):
    return "".join(c if c.isalnum() or c in "._- " else "_" for c in str(s)).strip().replace(" ", "_")

def cached_rows():
    done = set()
    for f in os.listdir(CACHE_DIR):
        if f.endswith("_meta.json") and f.startswith("row_"):
            try:
                idx = int(f.split("_")[1].split(".")[0])
                done.add(idx)
            except:
                pass
    return done

def append_log(row_index, street, status, filename, http_code, error_message=""):
    entry = {
        "timestamp": datetime.utcnow().isoformat() + "Z",
        "row_index": row_index,
        "street": street,
        "status": status,
        "filename": filename,
        "http_code": http_code,
        "error_message": error_message
    }
    df = pd.DataFrame([entry])
    df.to_csv(LOG_FILE, mode="a", index=False, header=not os.path.exists(LOG_FILE))

def save_response(name_base, data, meta):
    with open(os.path.join(CACHE_DIR, f"{name_base}.json"), "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)
    with open(os.path.join(CACHE_DIR, f"{name_base}_meta.json"), "w", encoding="utf-8") as f:
        json.dump(meta, f, indent=2)

def is_address_valid_before_api(row):
    # Required fields
    required = ["Number", "Street", "Suburb", "PostalCode"]

    # Check missing
    for f in required:
        if f not in row or pd.isna(row[f]) or str(row[f]).strip() == "":
            return False

    # Street number cannot be empty or non-numeric
    num = str(row["Number"]).strip()
    if not num.isdigit():
        return False

    # ZIP must be length ≥ 5 and numeric
    zip_ = str(row["PostalCode"]).replace(".0", "").strip()
    if not zip_.isdigit() or len(zip_) < 5:
        return False

    # Street must have at least 2 chars
    if len(str(row["Street"]).strip()) < 2:
        return False

    return True


# ========= MAIN PROCESSING ========= #

def main():
    df = pd.read_csv(INPUT_CSV)

    # NEW ======================================================
    invalid_rows = []  # collect invalid rows here
    # ==========================================================

    done_rows = cached_rows()
    print(f"Found {len(done_rows)} previously processed rows. Resuming automatically…")

    total_rows = len(df)

    for i, row in df.iterrows():

        # --- PREVALIDATION (NEW) ---
        if not is_address_valid_before_api(row):
            print(f"[{i}] ❌ Invalid address. Skipping BEFORE API call.")
            invalid_rows.append(row.to_dict())

            street_for_log = f"{row.get('Number','')} {row.get('Street','')}".strip()
            append_log(
                i,
                street_for_log,
                "pre_validation_fail",
                f"row_{i}",
                "",
                "Pre-validation rejected"
            )

            continue    
        
            
        if i in done_rows:
            print(f"[{i}] Skipping (cached).")
            continue

        number = str(row.get("Number", "")).strip()
        street_raw = str(row.get("Street", "")).strip()
        street = f"{number} {street_raw}".strip()
        city = str(row.get("Suburb", "")).strip()
        state = str(row.get("State", "CA")).strip()
        zipcode = str(row.get("PostalCode", "")).replace(".0", "").strip()
        input_secondary = get_input_secondary(row)

        params = {
            "auth-id": AUTH_ID,
            "auth-token": AUTH_TOKEN,
            "street": street,
            "city": city,
            "state": state,
            "zipcode": zipcode,
        }
        if input_secondary:
            params["secondary"] = input_secondary

        tentative_name = f"row_{i}"
        meta_stub = {
            "row_index": int(i),
            "sent_params": params,
            "timestamp": datetime.utcnow().isoformat() + "Z",
        }

        # --- RETRY MECHANISM ---
        retries = 3
        data = None
        while retries > 0:
            try:
                r = requests.get(SMARTY_URL, params=params, timeout=15)
                data = r.json() if r.content else None
                break
            except requests.exceptions.Timeout:
                retries -= 1
                wait = (4 - retries)
                print(f"[{i}] Timeout. Retrying in {wait}s…")
                time.sleep(wait)
        else:
            print(f"[{i}] FAILED after retries.")
            append_log(i, street, "timeout_failure", tentative_name, "TIMEOUT")
            continue

        # --- EXHAUSTION OR ERROR CHECK ---
        exhausted = False

        if r.status_code == 402:
            exhausted = True

        if isinstance(data, dict) and "errors" in data:
            exhausted = True

        if exhausted:
            print(f"❌ API exhausted at row {i}. Saving remaining rows to CSV…")
            remaining_df = df.iloc[i:]
            remaining_df.to_csv(REMAINING_CSV, index=False)

            # ALSO save invalid rows collected so far
            if invalid_rows:
                pd.DataFrame(invalid_rows).to_csv(INVALID_CSV, index=False)

            return

        # --- VALIDITY CHECK (NEW SECTION) ---
        if not isinstance(data, list) or len(data) == 0:
            print(f"[{i}] ⚠ INVALID JSON response → storing to invalid_rows.csv")
            invalid_rows.append(row)
            append_log(i, street, "invalid_json", tentative_name, r.status_code, "Empty or invalid JSON")
            continue

        # --- NAME FOR CACHE ---
        name_base = tentative_name
        first = data[0]

        dpb = first.get("delivery_point_barcode") or first.get("metadata", {}).get("delivery_point_barcode")
        if dpb:
            name_base = safe_filename(dpb)
        else:
            comps = first.get("components", {})
            pn = comps.get("primary_number") or number or i
            z = comps.get("zipcode") or zipcode
            name_base = safe_filename(f"{pn}_{z}_{i}")

        # Save the response JSON + meta
        save_response(name_base, data, meta_stub | {"status_code": r.status_code})
        append_log(i, street, "success", name_base, r.status_code)

        print(f"[{i}] Saved: {name_base}.json (HTTP {r.status_code})")

        time.sleep(0.5)

    # AFTER LOOP: save invalid rows
    if invalid_rows:
        pd.DataFrame(invalid_rows).to_csv(INVALID_CSV, index=False)

    print("🎉 Completed all rows successfully!")

if __name__ == "__main__":
    main()


Found 0 previously processed rows. Resuming automatically…
[0] Saved: 903016478011.json (HTTP 200)
[1] Saved: 903014211135.json (HTTP 200)
[2] Saved: 903014211171.json (HTTP 200)
[3] Saved: 903014211216.json (HTTP 200)
[4] Saved: 903014211252.json (HTTP 200)
[5] Saved: 903014211298.json (HTTP 200)
[6] Saved: 903014211333.json (HTTP 200)
[7] Saved: 903014211351.json (HTTP 200)
[8] Saved: 903014277016.json (HTTP 200)
[9] ❌ Invalid address. Skipping BEFORE API call.
[10] ❌ Invalid address. Skipping BEFORE API call.
[11] Saved: 903014223259.json (HTTP 200)
[12] Saved: 903014223277.json (HTTP 200)
[13] Saved: 903016475735.json (HTTP 200)
[14] Saved: 903014223330.json (HTTP 200)
[15] ⚠ INVALID JSON response → storing to invalid_rows.csv
[16] Saved: 903012707128.json (HTTP 200)
[17] Saved: 903016253010.json (HTTP 200)
[18] Saved: 903016242733.json (HTTP 200)
[19] Saved: 903016246739.json (HTTP 200)
[20] Saved: 903016250013.json (HTTP 200)
[21] Saved: 903012711286.json (HTTP 200)
[22] Saved: 9

Find invalid json caches

In [7]:
import os
import json
import shutil
import pandas as pd
from glob import glob
from datetime import datetime

# Paths
CACHE_DIR = "json_cache"
INVALID_DIR = "invalid_cache"
INVALID_CSV = "invalid_rows.csv"

os.makedirs(INVALID_DIR, exist_ok=True)

def looks_like_street_named_file(filename):
    """Check if filename follows street_index pattern rather than DPB pattern."""
    # DPB pattern is usually numeric or has specific length
    if filename.startswith("row_") or filename.endswith("_meta.json"):
        return False
    
    # Look for street patterns (words with spaces/underscores)
    parts = filename.replace(".json","").split("_")
    if len(parts) >= 2:
        # Check if last part is numeric (likely the index)
        if parts[-1].isdigit():
            # Check if earlier parts look like street words
            street_part = "_".join(parts[:-1])
            return any(c.isalpha() for c in street_part)
    return False

def load_json_safely(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except:
        return None

def extract_row_info(json_path, filename):
    """Extract original row info from JSON and meta files."""
    data = load_json_safely(json_path)
    meta_path = json_path.replace(".json", "_meta.json")
    meta = load_json_safely(meta_path)

    # Get original params from meta if available
    params = meta.get("sent_params", {}) if meta else {}
    
    row = {
        "Number": params.get("street", "").split()[0] if params.get("street") else "",
        "Street": " ".join(params.get("street", "").split()[1:]) if params.get("street") else "",
        "Suburb": params.get("city", ""),
        "State": params.get("state", ""),
        "PostalCode": params.get("zipcode", ""),
        "original_cache_file": filename,
        "moved_timestamp": datetime.now().isoformat()
    }
    return row

def main():
    # 1. Find invalid cache files
    invalid_files = []
    for f in os.listdir(CACHE_DIR):
        if f.endswith(".json") and not f.endswith("_meta.json"):
            if looks_like_street_named_file(f):
                invalid_files.append(f)
    
    if not invalid_files:
        print("No invalid cache files found.")
        return

    print(f"Found {len(invalid_files)} potentially invalid cache files.")
    
    # 2. Process each file
    invalid_rows = []
    moved_files = []
    
    for filename in invalid_files:
        src_path = os.path.join(CACHE_DIR, filename)
        meta_filename = filename.replace(".json", "_meta.json")
        src_meta_path = os.path.join(CACHE_DIR, meta_filename)
        
        # Extract row info
        row_info = extract_row_info(src_path, filename)
        invalid_rows.append(row_info)
        
        # Move files
        dst_path = os.path.join(INVALID_DIR, filename)
        dst_meta_path = os.path.join(INVALID_DIR, meta_filename)
        
        try:
            shutil.move(src_path, dst_path)
            if os.path.exists(src_meta_path):
                shutil.move(src_meta_path, dst_meta_path)
            moved_files.append(filename)
        except Exception as e:
            print(f"Error moving {filename}: {e}")

    # 3. Update invalid_rows.csv
    new_df = pd.DataFrame(invalid_rows)
    if os.path.exists(INVALID_CSV):
        existing_df = pd.read_csv(INVALID_CSV)
        combined_df = pd.concat([existing_df, new_df], ignore_index=True)
        combined_df.to_csv(INVALID_CSV, index=False)
    else:
        new_df.to_csv(INVALID_CSV, index=False)

    # 4. Print summary
    print("\n=== Summary ===")
    print(f"Total invalid files found: {len(invalid_files)}")
    print(f"Successfully moved: {len(moved_files)}")
    print(f"Added {len(invalid_rows)} rows to {INVALID_CSV}")
    print("\nMoved files are now in:", INVALID_DIR)
    print("Original address details added to:", INVALID_CSV)

if __name__ == "__main__":
    main()

Found 47 potentially invalid cache files.

=== Summary ===
Total invalid files found: 47
Successfully moved: 47
Added 47 rows to invalid_rows.csv

Moved files are now in: invalid_cache
Original address details added to: invalid_rows.csv


Validating invalid_rows.csv


In [1]:
import pandas as pd
import re

INPUT_FILE = "invalid_rows.csv"
OUTPUT_VALID = "validated_rows.csv"
OUTPUT_INVALID = "still_invalid_rows.csv"

COMMON_SUFFIXES = {
    "St": "Street", "Ave": "Avenue", "Blvd": "Boulevard", "Dr": "Drive",
    "Ln": "Lane", "Rd": "Road", "Pl": "Place", "Ct": "Court",
    "Ter": "Terrace", "Cir": "Circle", "Pkwy": "Parkway", "Hwy": "Highway"
}

def normalize_street(street):
    if not isinstance(street, str):
        return ""
    s = re.sub(r"\s+", " ", street.strip())  # clean spacing
    parts = s.split()
    if parts and parts[-1] in COMMON_SUFFIXES:
        parts[-1] = COMMON_SUFFIXES[parts[-1]]
    return " ".join(parts)

def normalize_secondary(sec):
    if not isinstance(sec, str):
        return ""
    sec = sec.strip()
    if not sec:
        return ""
    sec = sec.replace("Unit ", "").replace("Apt ", "")
    return "Unit " + sec

def is_valid(row):
    number = str(row.get("Number", "")).strip()
    street = str(row.get("Street", "")).strip()
    city = str(row.get("Suburb", "")).strip()
    zipcode = str(row.get("PostalCode", "")).strip()
    state = str(row.get("State", "")).strip()

    # Basic sanity checks
    if not number or not street or not city or not zipcode:
        return False
    if not re.match(r"^\d{5}$", zipcode):
        return False
    if len(street.split()) < 1:
        return False
    return True

def main():
    df = pd.read_csv(INPUT_FILE)

    # Normalize columns
    df["Street"] = df["Street"].apply(normalize_street)
    df["ApartmentNumber"] = df["ApartmentNumber"].apply(normalize_secondary)
    df["State"] = df["State"].fillna("CA")

    valid_rows = []
    invalid_rows = []

    for _, row in df.iterrows():
        if is_valid(row):
            valid_rows.append(row)
        else:
            invalid_rows.append(row)

    pd.DataFrame(valid_rows).to_csv(OUTPUT_VALID, index=False)
    pd.DataFrame(invalid_rows).to_csv(OUTPUT_INVALID, index=False)

    print(f"✅ {len(valid_rows)} valid rows saved to {OUTPUT_VALID}")
    print(f"⚠ {len(invalid_rows)} invalid rows saved to {OUTPUT_INVALID}")

if __name__ == "__main__":
    main()


✅ 426 valid rows saved to validated_rows.csv
⚠ 8 invalid rows saved to still_invalid_rows.csv


Sending to API

In [3]:
import os
import time
import json
import requests
import pandas as pd
from datetime import datetime

# --- CONFIGURE ---
AUTH_ID = "fc20c9c5-366d-71ae-5c8f-da11aa762646"
AUTH_TOKEN = "rOOG4cFXIaOANVasFnu7"
SMARTY_URL = "https://us-street.api.smarty.com/street-address"

INPUT_CSV = "validated_rows_try.csv"
CACHE_DIR = "json_cache"
LOG_FILE = "process_log.csv"
REMAINING_CSV = "remaining_rows_new.csv"
INVALID_CSV = "invalid_rows_new.csv"   # NEW: store rows with invalid JSON responses

os.makedirs(CACHE_DIR, exist_ok=True)

SECONDARY_CANDIDATES = [
    "ApartmentNumber", "Apartment", "Apt", "Unit", "Secondary", "SecondaryUnit",
    "apt_number", "apt", "unit_number"
]

# ========= Helper Functions ========= #

def get_input_secondary(row):
    for k in SECONDARY_CANDIDATES:
        if k in row.index:
            v = row.get(k)
            if pd.notna(v) and str(v).strip():
                return str(v).strip()
    return ""

def safe_filename(s):
    return "".join(c if c.isalnum() or c in "._- " else "_" for c in str(s)).strip().replace(" ", "_")

def cached_rows():
    done = set()
    for f in os.listdir(CACHE_DIR):
        if f.endswith("_meta.json") and f.startswith("row_"):
            try:
                idx = int(f.split("_")[1].split(".")[0])
                done.add(idx)
            except:
                pass
    return done

def append_log(row_index, street, status, filename, http_code, error_message=""):
    entry = {
        "timestamp": datetime.utcnow().isoformat() + "Z",
        "row_index": row_index,
        "street": street,
        "status": status,
        "filename": filename,
        "http_code": http_code,
        "error_message": error_message
    }
    df = pd.DataFrame([entry])
    df.to_csv(LOG_FILE, mode="a", index=False, header=not os.path.exists(LOG_FILE))

def save_response(name_base, data, meta):
    with open(os.path.join(CACHE_DIR, f"{name_base}.json"), "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)
    with open(os.path.join(CACHE_DIR, f"{name_base}_meta.json"), "w", encoding="utf-8") as f:
        json.dump(meta, f, indent=2)

def is_address_valid_before_api(row):
    # Required fields
    required = ["Number", "Street", "Suburb", "PostalCode"]

    # Check missing
    for f in required:
        if f not in row or pd.isna(row[f]) or str(row[f]).strip() == "":
            return False

    # Street number cannot be empty or non-numeric
    num = str(row["Number"]).strip()
    if not num.isdigit():
        return False

    # ZIP must be length ≥ 5 and numeric
    zip_ = str(row["PostalCode"]).replace(".0", "").strip()
    if not zip_.isdigit() or len(zip_) < 5:
        return False

    # Street must have at least 2 chars
    if len(str(row["Street"]).strip()) < 2:
        return False

    return True


# ========= MAIN PROCESSING ========= #

def main():
    df = pd.read_csv(INPUT_CSV)

    # NEW ======================================================
    invalid_rows = []  # collect invalid rows here
    # ==========================================================

    done_rows = cached_rows()
    print(f"Found {len(done_rows)} previously processed rows. Resuming automatically…")

    total_rows = len(df)

    for i, row in df.iterrows():

        # --- PREVALIDATION (NEW) ---
        if not is_address_valid_before_api(row):
            print(f"[{i}] ❌ Invalid address. Skipping BEFORE API call.")
            invalid_rows.append(row.to_dict())

            street_for_log = f"{row.get('Number','')} {row.get('Street','')}".strip()
            append_log(
                i,
                street_for_log,
                "pre_validation_fail",
                f"row_{i}",
                "",
                "Pre-validation rejected"
            )

            continue    
        
            
        if i in done_rows:
            print(f"[{i}] Skipping (cached).")
            continue

        number = str(row.get("Number", "")).strip()
        street_raw = str(row.get("Street", "")).strip()
        street = f"{number} {street_raw}".strip()
        city = str(row.get("Suburb", "")).strip()
        state = str(row.get("State", "CA")).strip()
        zipcode = str(row.get("PostalCode", "")).replace(".0", "").strip()
        input_secondary = get_input_secondary(row)

        params = {
            "auth-id": AUTH_ID,
            "auth-token": AUTH_TOKEN,
            "street": street,
            "city": city,
            "state": state,
            "zipcode": zipcode,
        }
        if input_secondary:
            params["secondary"] = input_secondary

        tentative_name = f"row_{i}"
        meta_stub = {
            "row_index": int(i),
            "sent_params": params,
            "timestamp": datetime.utcnow().isoformat() + "Z",
        }

        # --- RETRY MECHANISM ---
        retries = 3
        data = None
        while retries > 0:
            try:
                r = requests.get(SMARTY_URL, params=params, timeout=15)
                data = r.json() if r.content else None
                break
            except requests.exceptions.Timeout:
                retries -= 1
                wait = (4 - retries)
                print(f"[{i}] Timeout. Retrying in {wait}s…")
                time.sleep(wait)
        else:
            print(f"[{i}] FAILED after retries.")
            append_log(i, street, "timeout_failure", tentative_name, "TIMEOUT")
            continue

        # --- EXHAUSTION OR ERROR CHECK ---
        exhausted = False

        if r.status_code == 402:
            exhausted = True

        if isinstance(data, dict) and "errors" in data:
            exhausted = True

        if exhausted:
            print(f"❌ API exhausted at row {i}. Saving remaining rows to CSV…")
            remaining_df = df.iloc[i:]
            remaining_df.to_csv(REMAINING_CSV, index=False)

            # ALSO save invalid rows collected so far
            if invalid_rows:
                pd.DataFrame(invalid_rows).to_csv(INVALID_CSV, index=False)

            return

        # --- VALIDITY CHECK (NEW SECTION) ---
        if not isinstance(data, list) or len(data) == 0:
            print(f"[{i}] ⚠ INVALID JSON response → storing to invalid_rows.csv")
            invalid_rows.append(row)
            append_log(i, street, "invalid_json", tentative_name, r.status_code, "Empty or invalid JSON")
            continue

        # --- NAME FOR CACHE ---
        name_base = tentative_name
        first = data[0]

        dpb = first.get("delivery_point_barcode") or first.get("metadata", {}).get("delivery_point_barcode")
        if dpb:
            name_base = safe_filename(dpb)
        else:
            comps = first.get("components", {})
            pn = comps.get("primary_number") or number or i
            z = comps.get("zipcode") or zipcode
            name_base = safe_filename(f"{pn}_{z}_{i}")

        # Save the response JSON + meta
        save_response(name_base, data, meta_stub | {"status_code": r.status_code})
        append_log(i, street, "success", name_base, r.status_code)

        print(f"[{i}] Saved: {name_base}.json (HTTP {r.status_code})")

        time.sleep(0.5)

    # AFTER LOOP: save invalid rows
    if invalid_rows:
        pd.DataFrame(invalid_rows).to_csv(INVALID_CSV, index=False)

    print("🎉 Completed all rows successfully!")

if __name__ == "__main__":
    main()


Found 0 previously processed rows. Resuming automatically…
[0] ❌ Invalid address. Skipping BEFORE API call.
[1] ❌ Invalid address. Skipping BEFORE API call.
[2] ⚠ INVALID JSON response → storing to invalid_rows.csv
🎉 Completed all rows successfully!
